# 模型构造

In [1]:
import torch
from torch import nn

class AlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv=nn.Sequential(
            nn.Conv2d(1,96,11,4),
            nn.ReLU(),
            nn.MaxPool2d(3,2),
            nn.Conv2d(96,256,5,1,2),
            nn.ReLU(),
            nn.MaxPool2d(3,2),
            nn.Conv2d(256,384,3,1,1),
            nn.ReLU(),
            nn.Conv2d(384,384,3,1,1),
            nn.ReLU(),
            nn.Conv2d(384,256,3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(3,2)
        )

        self.fc=nn.Sequential(
            nn.Linear(256*5*5,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096,10)
        )

    def forward(self,img):
        feature=self.conv(img)
        output=self.fc(feature.view(feature.shape[0],-1))
        return output
net=AlexNet()
print(net)

AlexNet(
  (conv): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=6400, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (

# 读取数据集

In [2]:
import torchvision

transform = torchvision.transforms.Compose(
    [torchvision.transforms.Resize(size=224),
    torchvision.transforms.ToTensor()]
)

#获取数据集
mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=True, download=True, transform=transform)
mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=False, download=True, transform=transform)
#读取数据集
batchSize=128
trainIter=torch.utils.data.DataLoader(mnist_train,batch_size=batchSize,shuffle=True,num_workers=8)
testIter=torch.utils.data.DataLoader(mnist_test,batch_size=batchSize,shuffle=True,num_workers=8)

# 评价

In [3]:
def evaluate_accuracy(data_iter, net):
    device=torch.device('cuda')
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        net.eval() # 评估模式
        for X, y in data_iter:
            acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().item()
            n += y.shape[0]
        net.train() # 改回训练模式 
    return acc_sum / n

# 训练

In [4]:
lr,epochsNum = 0.001,5
net=net.cuda()
loss=torch.nn.CrossEntropyLoss()
optimzer=torch.optim.Adam(net.parameters(),lr)

for epoch in range(epochsNum):
    train_l_sum=n=train_acc_sum=0
    for x,y in trainIter:
        x=x.cuda()
        y=y.cuda()
        yP=net(x)
        l=loss(yP,y)
        l.backward()
        optimzer.step()
        optimzer.zero_grad()
        train_l_sum += l.item()
        train_acc_sum += (yP.argmax(dim=1) == y).sum().item()
        n +=y.shape[0]
    print(epoch,train_l_sum/n,train_acc_sum/n,evaluate_accuracy(testIter,net))

0 0.0050765483026703195 0.7494333333333333 0.8565
1 0.002745510173837344 0.8701333333333333 0.8794
2 0.002377479326725006 0.8876666666666667 0.8893
3 0.002120009563987454 0.89945 0.8914
4 0.0019536696154624222 0.9082666666666667 0.8984
